# Toxikind: Models

In [35]:
# Runtime
import time

# OS I/O
import os
import pickle

# Data handling
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer

# Modeling
from sklearn.model_selection import cross_validate, cross_val_predict, GridSearchCV
from sklearn.metrics import f1_score, precision_recall_curve, classification_report, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Training Data

## Full Data

**Load features:**

In [2]:
path_x_train = "../data/X_train.csv"
X_train = pd.read_csv(path_x_train).set_index("ID")
X_train

,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
ID,,,,,,,,,,,,,,,,,,,,,
NCGC00178831-03,5.436720e-01,0.012041,0.815898,0.832518,0.244068,0.251849,0.122744,0.255645,0.247884,0.131926,...,0.028705,0.025992,0.084419,0.992430,0.967627,0.086494,0.022640,0.042745,0.000218,0.231025
NCGC00166114-03,1.268818e-01,0.076781,0.774278,0.816115,0.222113,0.217725,0.158927,0.222859,0.235265,0.126033,...,0.023329,0.009094,0.142494,0.991487,0.972305,0.070178,0.012238,0.059568,0.000336,0.258584
NCGC00263563-01,3.076931e-02,0.012270,0.807649,0.836259,0.495232,0.475479,0.312274,0.508771,0.503347,0.393865,...,0.147491,0.215437,0.174151,0.963958,0.913740,0.259019,0.156472,0.097238,0.000542,0.320086
NCGC00013058-02,7.168569e-01,0.010464,0.760780,0.886043,0.539479,0.508602,0.148179,0.491618,0.425273,0.263047,...,0.147532,0.142056,0.165596,0.976968,0.949863,0.227592,0.092400,0.090178,0.000596,0.291745
NCGC00167516-01,7.989701e-02,0.011206,0.796400,0.941583,0.742331,0.736470,0.295701,0.712026,0.641167,0.352710,...,0.461955,0.460399,0.622595,0.825232,0.702753,0.536326,0.439447,0.260218,0.000796,0.500554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCGC00261292-01,1.428572e-01,0.020621,0.749906,0.669640,0.097777,0.100133,0.012882,0.092409,0.088271,0.038183,...,0.003261,0.004034,0.055917,0.998427,0.988861,0.030003,0.002463,0.013849,0.000069,0.157600
NCGC00261245-01,1.193182e-01,0.016474,0.772778,0.753669,0.223264,0.244135,0.098540,0.214848,0.208197,0.054503,...,0.027008,0.034580,0.069424,0.995076,0.981965,0.096647,0.020009,0.035527,0.000232,0.228288
NCGC00260828-01,9.818000e-08,0.007195,0.766779,0.813525,0.351043,0.324683,0.066131,0.333114,0.291330,0.238414,...,0.097991,0.123973,0.195661,0.978034,0.915763,0.260806,0.085466,0.103675,0.000227,0.514170


**Load targets:**

In [3]:
path_y_train = "../data/y_train.csv"
y_train = pd.read_csv(path_y_train).set_index("ID")
y_train

,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
ID,,,,,,,,,,,,
NCGC00178831-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
NCGC00166114-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
NCGC00263563-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
NCGC00013058-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
NCGC00167516-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
NCGC00261292-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCGC00261245-01,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
NCGC00260828-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Without NAs For Each Target

### 1. NR.AhR

In [4]:
# Create single target without NAs
y_train_ahr = y_train[["NR.AhR"]].dropna()

# Inner merge with features:
train_ahr = y_train_ahr.merge(X_train, how="inner", on="ID")

# Inspect
print(train_ahr.info())

# Split again
X_train_ahr = train_ahr.drop(columns=["NR.AhR"])
y_train_ahr = train_ahr["NR.AhR"]

<class 'pandas.core.frame.DataFrame'>
Index: 8441 entries, NCGC00255644-01 to NCGC00261465-01
Columns: 802 entries, NR.AhR to rygr
dtypes: float64(802)
memory usage: 51.7+ MB
None


### 2. NR.AR

In [5]:
# Create single target without NAs
y_train_ar = y_train[["NR.AR"]].dropna()

# Inner merge with features:
train_ar = y_train_ar.merge(X_train, how="inner", on="ID")

# Inspect
print(train_ar.info())

# Split again
X_train_ar = train_ar.drop(columns=["NR.AR"])
y_train_ar = train_ar["NR.AR"]

<class 'pandas.core.frame.DataFrame'>
Index: 9654 entries, NCGC00167516-01 to NCGC00261465-01
Columns: 802 entries, NR.AR to rygr
dtypes: float64(802)
memory usage: 59.1+ MB
None


### 3. NR.AR.LBD

In [6]:
# Create single target without NAs
y_train_lbd = y_train[["NR.AR.LBD"]].dropna()

# Inner merge with features:
train_lbd = y_train_lbd.merge(X_train, how="inner", on="ID")

# Inspect
print(train_lbd.info())

# Split again
X_train_lbd = train_lbd.drop(columns=["NR.AR.LBD"])
y_train_lbd = train_lbd["NR.AR.LBD"]

<class 'pandas.core.frame.DataFrame'>
Index: 8852 entries, NCGC00094891-01 to NCGC00261465-01
Columns: 802 entries, NR.AR.LBD to rygr
dtypes: float64(802)
memory usage: 54.2+ MB
None


### 4. NR.Aromatase

In [7]:
# Create single target without NAs
y_train_aromatase = y_train[["NR.Aromatase"]].dropna()

# Inner merge with features:
train_aromatase = y_train_aromatase.merge(X_train, how="inner", on="ID")

# Inspect
print(train_aromatase.info())

# Split again
X_train_aromatase = train_aromatase.drop(columns=["NR.Aromatase"])
y_train_aromatase = train_aromatase["NR.Aromatase"]

<class 'pandas.core.frame.DataFrame'>
Index: 7440 entries, NCGC00258497-01 to NCGC00261465-01
Columns: 802 entries, NR.Aromatase to rygr
dtypes: float64(802)
memory usage: 45.6+ MB
None


### 5. NR.ER

In [8]:
# Create single target without NAs
y_train_er = y_train[["NR.ER"]].dropna()

# Inner merge with features:
train_er = y_train_er.merge(X_train, how="inner", on="ID")

# Inspect
print(train_er.info())

# Split again
X_train_er = train_er.drop(columns=["NR.ER"])
y_train_er = train_er["NR.ER"]

<class 'pandas.core.frame.DataFrame'>
Index: 7962 entries, NCGC00161831-02 to NCGC00260687-01
Columns: 802 entries, NR.ER to rygr
dtypes: float64(802)
memory usage: 48.8+ MB
None


### 6. NR.ER.LBD

In [9]:
# Create single target without NAs
y_train_er_lbd = y_train[["NR.ER.LBD"]].dropna()

# Inner merge with features:
train_er_lbd = y_train_er_lbd.merge(X_train, how="inner", on="ID")

# Inspect
print(train_er_lbd.info())

# Split again
X_train_er_lbd = train_er_lbd.drop(columns=["NR.ER.LBD"])
y_train_er_lbd = train_er_lbd["NR.ER.LBD"]

<class 'pandas.core.frame.DataFrame'>
Index: 9040 entries, NCGC00095843-01 to NCGC00261465-01
Columns: 802 entries, NR.ER.LBD to rygr
dtypes: float64(802)
memory usage: 55.4+ MB
None


### 7. NR.PPAR.gamma

In [10]:
# Create single target without NAs
y_train_ppar_gamma = y_train[["NR.PPAR.gamma"]].dropna()

# Inner merge with features:
train_ppar_gamma = y_train_ppar_gamma.merge(X_train, how="inner", on="ID")

# Inspect
print(train_ppar_gamma.info())

# Split again
X_train_ppar_gamma = train_ppar_gamma.drop(columns=["NR.PPAR.gamma"])
y_train_ppar_gamma = train_ppar_gamma["NR.PPAR.gamma"]

<class 'pandas.core.frame.DataFrame'>
Index: 8451 entries, NCGC00091441-01 to NCGC00261465-01
Columns: 802 entries, NR.PPAR.gamma to rygr
dtypes: float64(802)
memory usage: 51.8+ MB
None


### 8. SR.ARE

In [11]:
# Create single target without NAs
y_train_are = y_train[["SR.ARE"]].dropna()

# Inner merge with features:
train_are = y_train_are.merge(X_train, how="inner", on="ID")

# Inspect
print(train_are.info())

# Split again
X_train_are = train_are.drop(columns=["SR.ARE"])
y_train_are = train_are["SR.ARE"]

<class 'pandas.core.frame.DataFrame'>
Index: 7401 entries, NCGC00095016-05 to NCGC00261465-01
Columns: 802 entries, SR.ARE to rygr
dtypes: float64(802)
memory usage: 45.3+ MB
None


### 9. SR.ATAD5

In [12]:
# Create single target without NAs
y_train_atad5 = y_train[["SR.ATAD5"]].dropna()

# Inner merge with features:
train_atad5 = y_train_atad5.merge(X_train, how="inner", on="ID")

# Inspect
print(train_atad5.info())

# Split again
X_train_atad5 = train_atad5.drop(columns=["SR.ATAD5"])
y_train_atad5 = train_atad5["SR.ATAD5"]

<class 'pandas.core.frame.DataFrame'>
Index: 9363 entries, NCGC00094121-01 to NCGC00261465-01
Columns: 802 entries, SR.ATAD5 to rygr
dtypes: float64(802)
memory usage: 57.4+ MB
None


### 10. SR.HSE

In [13]:
# Create single target without NAs
y_train_hse = y_train[["SR.HSE"]].dropna()

# Inner merge with features:
train_hse = y_train_hse.merge(X_train, how="inner", on="ID")

# Inspect
print(train_hse.info())

# Split again
X_train_hse = train_hse.drop(columns=["SR.HSE"])
y_train_hse = train_hse["SR.HSE"]

<class 'pandas.core.frame.DataFrame'>
Index: 8417 entries, NCGC00178831-03 to NCGC00261465-01
Columns: 802 entries, SR.HSE to rygr
dtypes: float64(802)
memory usage: 51.6+ MB
None


### 11. SR.MMP

In [14]:
# Create single target without NAs
y_train_mmp = y_train[["SR.MMP"]].dropna()

# Inner merge with features:
train_mmp = y_train_mmp.merge(X_train, how="inner", on="ID")

# Inspect
print(train_mmp.info())

# Split again
X_train_mmp = train_mmp.drop(columns=["SR.MMP"])
y_train_mmp = train_mmp["SR.MMP"]

<class 'pandas.core.frame.DataFrame'>
Index: 7558 entries, NCGC00165865-01 to NCGC00261465-01
Columns: 802 entries, SR.MMP to rygr
dtypes: float64(802)
memory usage: 46.3+ MB
None


### 12. SR.p53

In [15]:
# Create single target without NAs
y_train_p53 = y_train[["SR.p53"]].dropna()

# Inner merge with features:
train_p53 = y_train_p53.merge(X_train, how="inner", on="ID")

# Inspect
print(train_p53.info())

# Split again
X_train_p53 = train_p53.drop(columns=["SR.p53"])
y_train_p53 = train_p53["SR.p53"]

<class 'pandas.core.frame.DataFrame'>
Index: 8903 entries, NCGC00166288-01 to NCGC00261465-01
Columns: 802 entries, SR.p53 to rygr
dtypes: float64(802)
memory usage: 54.5+ MB
None


# Test Data

## Full Data

**Load features:**

In [16]:
path_x_test = "../data/X_test.csv"
X_test = pd.read_csv(path_x_test).set_index("ID")
X_test

,AW,AWeight,Arto,BertzCT,Chi0,Chi1,Chi10,Chi2,Chi3,Chi3c,...,W3D,W3DH,WNSA1,WNSA2,WNSA3,WPSA1,WPSA2,WPSA3,grav,rygr
ID,,,,,,,,,,,,,,,,,,,,,
NCGC00261900-01,2.612482e-01,0.009436,0.834646,0.841727,0.394202,0.403771,0.300542,0.408977,0.398087,0.229952,...,0.103505,0.119139,0.142278,0.979173,0.932708,0.230205,0.098362,0.094579,0.000364,0.347710
NCGC00260869-01,8.333336e-02,0.043783,0.812523,0.754532,0.172691,0.172555,0.097883,0.187859,0.182665,0.135654,...,0.013413,0.007371,0.076107,0.996066,0.972912,0.065423,0.009872,0.055111,0.000187,0.232328
NCGC00261776-01,3.074000e-08,0.007837,0.886389,0.789065,0.155035,0.171825,0.176321,0.168661,0.186405,0.067600,...,0.011447,0.008864,0.068405,0.997175,0.982732,0.052118,0.006282,0.023628,0.000112,0.227702
NCGC00261380-01,8.000004e-02,0.017566,0.779903,0.732086,0.177179,0.186014,0.063751,0.171348,0.161561,0.063016,...,0.015035,0.012926,0.093085,0.995702,0.977549,0.059355,0.007999,0.025659,0.000143,0.227832
NCGC00261842-01,3.838000e-08,0.022434,0.782527,0.742158,0.178172,0.173317,0.033886,0.169590,0.151955,0.105279,...,0.012993,0.012214,0.098637,0.993249,0.955067,0.057505,0.011489,0.023847,0.000140,0.226269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCGC00357168-01,1.000000e-08,0.038929,0.599925,0.246331,0.043519,0.038314,0.000000,0.022882,0.013918,0.000000,...,0.000256,0.000578,0.018569,0.999713,0.993413,0.013657,0.000617,0.005536,0.000013,0.111147
NCGC00357283-01,2.714000e-08,0.013148,0.749906,0.527482,0.117176,0.115798,0.005907,0.095839,0.080535,0.030828,...,0.004334,0.007400,0.037757,0.998458,0.979364,0.045173,0.005385,0.017875,0.000070,0.183334
NCGC00357210-01,9.050000e-09,0.033198,0.749906,0.573813,0.055716,0.052455,0.000000,0.048756,0.045081,0.023726,...,0.000517,0.000408,0.025334,0.999671,0.994989,0.011548,0.000437,0.005116,0.000028,0.105349


**Load targets:**

In [17]:
path_y_test = "../data/y_test.csv"
y_test = pd.read_csv(path_y_test).set_index("ID")
y_test

,NR.AhR,NR.AR,NR.AR.LBD,NR.Aromatase,NR.ER,NR.ER.LBD,NR.PPAR.gamma,SR.ARE,SR.ATAD5,SR.HSE,SR.MMP,SR.p53
ID,,,,,,,,,,,,
NCGC00261900-01,0.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0
NCGC00260869-01,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCGC00261776-01,1.0,1.0,0.0,NaN,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
NCGC00261380-01,NaN,0.0,NaN,1.0,0.0,NaN,NaN,1.0,0.0,NaN,0.0,NaN
NCGC00261842-01,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
NCGC00357168-01,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
NCGC00357283-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NCGC00357210-01,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0


## Data Without NAs For Each Target

### 1. NR.AhR

In [18]:
# Create single target without NAs
y_test_ahr = y_test[["NR.AhR"]].dropna()

# Inner merge with features:
test_ahr = y_test_ahr.merge(X_test, how="inner", on="ID")

# Inspect
print(test_ahr.info())

# Split again
X_test_ahr = test_ahr.drop(columns=["NR.AhR"])
y_test_ahr = test_ahr["NR.AhR"]

<class 'pandas.core.frame.DataFrame'>
Index: 610 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, NR.AhR to rygr
dtypes: float64(802)
memory usage: 3.7+ MB
None


### 2. NR.AR

In [19]:
# Create single target without NAs
y_test_ar = y_test[["NR.AR"]].dropna()

# Inner merge with features:
test_ar = y_test_ar.merge(X_test, how="inner", on="ID")

# Inspect
print(test_ar.info())

# Split again
X_test_ar = test_ar.drop(columns=["NR.AR"])
y_test_ar = test_ar["NR.AR"]

<class 'pandas.core.frame.DataFrame'>
Index: 586 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, NR.AR to rygr
dtypes: float64(802)
memory usage: 3.6+ MB
None


### 3. NR.AR.LBD

In [20]:
# Create single target without NAs
y_test_lbd = y_test[["NR.AR.LBD"]].dropna()

# Inner merge with features:
test_lbd = y_test_lbd.merge(X_test, how="inner", on="ID")

# Inspect
print(test_lbd.info())

# Split again
X_test_lbd = test_lbd.drop(columns=["NR.AR.LBD"])
y_test_lbd = test_lbd["NR.AR.LBD"]

<class 'pandas.core.frame.DataFrame'>
Index: 582 entries, NCGC00261776-01 to NCGC00357144-01
Columns: 802 entries, NR.AR.LBD to rygr
dtypes: float64(802)
memory usage: 3.6+ MB
None


### 4. NR.Aromatase

In [21]:
# Create single target without NAs
y_test_aromatase = y_test[["NR.Aromatase"]].dropna()

# Inner merge with features:
test_aromatase = y_test_aromatase.merge(X_test, how="inner", on="ID")

# Inspect
print(test_aromatase.info())

# Split again
X_test_aromatase = test_aromatase.drop(columns=["NR.Aromatase"])
y_test_aromatase = test_aromatase["NR.Aromatase"]

<class 'pandas.core.frame.DataFrame'>
Index: 528 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, NR.Aromatase to rygr
dtypes: float64(802)
memory usage: 3.2+ MB
None


### 5. NR.ER

In [22]:
# Create single target without NAs
y_test_er = y_test[["NR.ER"]].dropna()

# Inner merge with features:
test_er = y_test_er.merge(X_test, how="inner", on="ID")

# Inspect
print(test_er.info())

# Split again
X_test_er = test_er.drop(columns=["NR.ER"])
y_test_er = test_er["NR.ER"]

<class 'pandas.core.frame.DataFrame'>
Index: 516 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, NR.ER to rygr
dtypes: float64(802)
memory usage: 3.2+ MB
None


### 6. NR.ER.LBD

In [23]:
# Create single target without NAs
y_test_er_lbd = y_test[["NR.ER.LBD"]].dropna()

# Inner merge with features:
test_er_lbd = y_test_er_lbd.merge(X_test, how="inner", on="ID")

# Inspect
print(test_er_lbd.info())

# Split again
X_test_er_lbd = test_er_lbd.drop(columns=["NR.ER.LBD"])
y_test_er_lbd = test_er_lbd["NR.ER.LBD"]

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, NCGC00261900-01 to NCGC00357118-01
Columns: 802 entries, NR.ER.LBD to rygr
dtypes: float64(802)
memory usage: 3.7+ MB
None


### 7. NR.PPAR.gamma

In [24]:
# Create single target without NAs
y_test_ppar_gamma = y_test[["NR.PPAR.gamma"]].dropna()

# Inner merge with features:
test_ppar_gamma = y_test_ppar_gamma.merge(X_test, how="inner", on="ID")

# Inspect
print(test_ppar_gamma.info())

# Split again
X_test_ppar_gamma = test_ppar_gamma.drop(columns=["NR.PPAR.gamma"])
y_test_ppar_gamma = test_ppar_gamma["NR.PPAR.gamma"]

<class 'pandas.core.frame.DataFrame'>
Index: 605 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, NR.PPAR.gamma to rygr
dtypes: float64(802)
memory usage: 3.7+ MB
None


### 8. SR.ARE

In [25]:
# Create single target without NAs
y_test_are = y_test[["SR.ARE"]].dropna()

# Inner merge with features:
test_are = y_test_are.merge(X_test, how="inner", on="ID")

# Inspect
print(test_are.info())

# Split again
X_test_are = test_are.drop(columns=["SR.ARE"])
y_test_are = test_are["SR.ARE"]

<class 'pandas.core.frame.DataFrame'>
Index: 555 entries, NCGC00260869-01 to NCGC00357144-01
Columns: 802 entries, SR.ARE to rygr
dtypes: float64(802)
memory usage: 3.4+ MB
None


### 9. SR.ATAD5

In [26]:
# Create single target without NAs
y_test_atad5 = y_test[["SR.ATAD5"]].dropna()

# Inner merge with features:
test_atad5 = y_test_atad5.merge(X_test, how="inner", on="ID")

# Inspect
print(test_atad5.info())

# Split again
X_test_atad5 = test_atad5.drop(columns=["SR.ATAD5"])
y_test_atad5 = test_atad5["SR.ATAD5"]

<class 'pandas.core.frame.DataFrame'>
Index: 622 entries, NCGC00261900-01 to NCGC00357118-01
Columns: 802 entries, SR.ATAD5 to rygr
dtypes: float64(802)
memory usage: 3.8+ MB
None


### 10. SR.HSE

In [27]:
# Create single target without NAs
y_test_hse = y_test[["SR.HSE"]].dropna()

# Inner merge with features:
test_hse = y_test_hse.merge(X_test, how="inner", on="ID")

# Inspect
print(test_hse.info())

# Split again
X_test_hse = test_hse.drop(columns=["SR.HSE"])
y_test_hse = test_hse["SR.HSE"]

<class 'pandas.core.frame.DataFrame'>
Index: 610 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, SR.HSE to rygr
dtypes: float64(802)
memory usage: 3.7+ MB
None


### 11. SR.MMP

In [28]:
# Create single target without NAs
y_test_mmp = y_test[["SR.MMP"]].dropna()

# Inner merge with features:
test_mmp = y_test_mmp.merge(X_test, how="inner", on="ID")

# Inspect
print(test_mmp.info())

# Split again
X_test_mmp = test_mmp.drop(columns=["SR.MMP"])
y_test_mmp = test_mmp["SR.MMP"]

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, NCGC00260869-01 to NCGC00357144-01
Columns: 802 entries, SR.MMP to rygr
dtypes: float64(802)
memory usage: 3.3+ MB
None


### 12. SR.p53

In [29]:
# Create single target without NAs
y_test_p53 = y_test[["SR.p53"]].dropna()

# Inner merge with features:
test_p53 = y_test_p53.merge(X_test, how="inner", on="ID")

# Inspect
print(test_p53.info())

# Split again
X_test_p53 = test_p53.drop(columns=["SR.p53"])
y_test_p53 = test_p53["SR.p53"]

<class 'pandas.core.frame.DataFrame'>
Index: 616 entries, NCGC00261900-01 to NCGC00357144-01
Columns: 802 entries, SR.p53 to rygr
dtypes: float64(802)
memory usage: 3.8+ MB
None


# Gradient Boost

## 0. Hyperparameters, Definition & Instantiation, Cross Validation Function

In [38]:
# Hyperparameters
n_estimators = 300
learning_rate= 0.1
max_depth = 4

# Paths
path_model_GBC = "../models"

In [31]:
# Definition and Instantiation
# Instantiate
model_GBC = GradientBoostingClassifier(
    n_estimators=n_estimators, 
    learning_rate=learning_rate,
    max_depth=max_depth
    )

In [ ]:
def cross_validate_classifier(model, X_test, y_test, cv=5) -> dict:
    """
    This function calls "sklearn.model_selection.cross_validate" with
    standard one-number metrics for a classifier and returns a dictionary
    with the validation results.
    """
    

## 1. NR.AhR

In [34]:
# Fit model
model_GBC_ahr = model_GBC.fit(X_train_ahr, y_train_ahr)

In [39]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_ahr = f"{path_model_GBC}/model_GBC_ahr-{timestamp}.pkl"
with open(path_model_GBC_ahr, "wb") as file:
    pickle.dump(model_GBC_ahr, file)

## 2. NR.AR

In [41]:
# Fit model
model_GBC_ar = model_GBC.fit(X_train_ar, y_train_ar)

In [44]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_ar = f"{path_model_GBC}/model_GBC_ar-{timestamp}.pkl"
with open(path_model_GBC_ar, "wb") as file:
    pickle.dump(model_GBC_ar, file)

## 3. NR.AR.LBD

In [45]:
# Fit model
model_GBC_lbd = model_GBC.fit(X_train_lbd, y_train_lbd)

In [47]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_lbd = f"{path_model_GBC}/model_GBC_lbd-{timestamp}.pkl"
with open(path_model_GBC_lbd, "wb") as file:
    pickle.dump(model_GBC_lbd, file)

## 4. NR.Aromatase

In [48]:
# Fit model
model_GBC_aromatase = model_GBC.fit(X_train_aromatase, y_train_aromatase)

In [49]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_aromatase = f"{path_model_GBC}/model_GBC_aromatase-{timestamp}.pkl"
with open(path_model_GBC_aromatase, "wb") as file:
    pickle.dump(model_GBC_aromatase, file)

## 5. NR.ER

In [50]:
# Fit model
model_GBC_er = model_GBC.fit(X_train_er, y_train_er)

In [51]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_er = f"{path_model_GBC}/model_GBC_er-{timestamp}.pkl"
with open(path_model_GBC_er, "wb") as file:
    pickle.dump(model_GBC_er, file)

## 6. NR.ER.LBD

In [52]:
# Fit model
model_GBC_er_lbd = model_GBC.fit(X_train_er_lbd, y_train_er_lbd)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_er_lbd = f"{path_model_GBC}/model_GBC_er_lbd-{timestamp}.pkl"
with open(path_model_GBC_er_lbd, "wb") as file:
    pickle.dump(model_GBC_er_lbd, file)

## 7. NR.PPAR.gamma

In [ ]:
# Fit model
model_GBC_ppar_gamma = model_GBC.fit(X_train_ppar_gamma, y_train_ppar_gamma)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_ppar_gamma = f"{path_model_GBC}/model_GBC_ppar_gamma-{timestamp}.pkl"
with open(path_model_GBC_ppar_gamma, "wb") as file:
    pickle.dump(model_GBC_ppar_gamma, file)

## 8. SR.ARE

In [ ]:
# Fit model
model_GBC_are = model_GBC.fit(X_train_are, y_train_are)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_are = f"{path_model_GBC}/model_GBC_are-{timestamp}.pkl"
with open(path_model_GBC_are, "wb") as file:
    pickle.dump(model_GBC_are, file)

## 9. SR.ATAD5

In [ ]:
# Fit model
model_GBC_atad5 = model_GBC.fit(X_train_atad5, y_train_atad5)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_atad5 = f"{path_model_GBC}/model_GBC_atad5-{timestamp}.pkl"
with open(path_model_GBC_atad5, "wb") as file:
    pickle.dump(model_GBC_atad5, file)

## 10. SR.HSE

In [ ]:
# Fit model
model_GBC_hse = model_GBC.fit(X_train_hse, y_train_hse)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_hse = f"{path_model_GBC}/model_GBC_hse-{timestamp}.pkl"
with open(path_model_GBC_hse, "wb") as file:
    pickle.dump(model_GBC_hse, file)

## 11. SR.MMP

In [ ]:
# Fit model
model_GBC_mmp = model_GBC.fit(X_train_mmp, y_train_mmp)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_mmp = f"{path_model_GBC}/model_GBC_mmp-{timestamp}.pkl"
with open(path_model_GBC_mmp, "wb") as file:
    pickle.dump(model_GBC_mmp, file)

## 12. SR.p53

In [ ]:
# Fit model
model_GBC_p53 = model_GBC.fit(X_train_p53, y_train_p53)

In [ ]:
# Save fitted model
timestamp = time.strftime("%Y%m%d-%H%M%S")

path_model_GBC_p53 = f"{path_model_GBC}/model_GBC_p53-{timestamp}.pkl"
with open(path_model_GBC_p53, "wb") as file:
    pickle.dump(model_GBC_p53, file)